In [28]:
import tsv
import numpy as np
import pandas as pd

In [29]:
root_path = "../data/poly_ro/"

In [43]:
# Poly_truth.tsv: true counts for each transcript
poly_truth = open(root_path + "poly_truth.tsv")
lines = poly_truth.readlines()
poly_truth.close()

# unlike poly_mo, first line does not have column labels
poly_truth = [['transcript_id','count']]
poly_truth.extend(line[:-1].split('\t') for line in lines)

df_poly_truth = pd.DataFrame.from_records(poly_truth[1:], columns=poly_truth[0])

In [44]:
df_poly_truth['transcript_id']=df_poly_truth['transcript_id'].astype(str)
df_poly_truth['count']=df_poly_truth['count'].astype(int)

In [45]:
quant_bootstraps = tsv.TsvReader(open(root_path + "quant_bootstraps.tsv"))

quant_boot = [line for line in quant_bootstraps]

In [46]:
df_quant_boot = pd.DataFrame.from_records(quant_boot[1:], columns=quant_boot[0])

In [47]:
intersect_ids = set(df_quant_boot.columns) & set(df_poly_truth.transcript_id)

sort_qb = []
use_id = []
for id in intersect_ids:
    listed = list(df_quant_boot[id])
    listed.sort()
    use_id.append(id)
    sort_qb.append(listed)
sort_qb = list(map(list,zip(*sort_qb)))

In [48]:
df_qb_sorted = pd.DataFrame.from_records(sort_qb, columns=use_id)

In [49]:
df_poly_truth = df_poly_truth.set_index(['transcript_id'])

sum = len(sort_qb)
percent2dot5 = df_qb_sorted.loc[int(sum*0.025)-1]
percent97dot5 = df_qb_sorted.loc[int(sum*0.975)-1]

167.344763292
167.344763292


In [50]:
true_id = []
false_id = []
for id in use_id:
    down = float(percent2dot5[id])
    up = float(percent97dot5[id])
    true_count = float(df_poly_truth.loc[id])
    if down < true_count < up: 
        true_id.append(id)
    else: 
        false_id.append(id)

In [52]:
id_qb = df_quant_boot.columns
extend_true = list(set(id_qb).difference(set(truth_id)))
true_id.extend(extend_true)

In [53]:
all_id = true_id[:]
all_id.extend(false_id)

In [55]:
label = [1 if i < len(true_id) else 0 for i in range(len(true_id) + len(false_id))]
    
labeled_id = [all_id,label]
labeled = list(map(list,zip(*labeled_id)))

In [56]:
quant_file = open(root_path + "quant.sf")
lines = quant_file.readlines()
quant_file.close()
quant = [line[:-1].split('\t') for line in lines]

In [57]:
df_quant = pd.DataFrame.from_records(quant[1:], columns=quant[0])

df_quant.Name = df_quant.Name.astype(str)
df_quant.Length = df_quant.Length.astype(int)
df_quant.EffectiveLength = df_quant.EffectiveLength.astype(float)
df_quant.TPM = df_quant.TPM.astype(float)
df_quant.NumReads = df_quant.NumReads.astype(float)

In [58]:
df_labeled_id = pd.DataFrame.from_records(labeled, columns=['Name','label'])
df_labeled_id.Name = df_labeled_id.Name.astype(str)
df_labeled = df_labeled_id.merge(df_quant, on='Name')

label = df_labeled.pop('label')
df_labeled.insert(5,'label',label)

data = df_labeled

In [59]:
data.groupby([data.label]).mean()

,Length,EffectiveLength,TPM,NumReads,label
label,,,,,
0,2320.422083,2121.332135,41.485945,1378.079501,0.0
1,1901.681056,1703.008206,0.154861,6.681882,1.0


In [60]:
data.groupby([data.label]).max()

,Name,Length,EffectiveLength,TPM,NumReads
label,,,,,
0,ENST00000610279,101518,101318.991,23356.420222,1.109005e+06
1,ENST00000610276,109224,109024.991,2435.717783,3.110509e+04


In [61]:
data.groupby([data.label]).min()

,Name,Length,EffectiveLength,TPM,NumReads
label,,,,,
0,ENST00000000233,82,10.861,0.0,0.0
1,ENST00000000412,21,9.784,0.0,0.0


In [62]:
data.groupby([data.label]).std()

,Length,EffectiveLength,TPM,NumReads
label,,,,
0,2302.710811,2302.636007,396.200319,13629.239858
1,2059.808430,2059.407366,9.489371,152.179457
